In [1]:
import numpy, math, time
from JokeFunc import LinSys
from importlib import reload
LinSys = reload(LinSys)

# Find linear system (Ax = B) of 2D problem

In [2]:
#Problem function F(u)=f(x,y)
def f(x,y):
    return 4*(math.pi)*(math.pi)*math.sin(2*math.pi*x)*y*y*(1-y)*(1-y)-math.sin(2*math.pi*x)*(2-12*y+12*y*y)

#Boundary Conditions
def bf(x,y):
    return 0

#Setup variables
nx = 70 #number_intervals for x
ax = 0
bx = 1
hx = (bx-ax)/nx
hx2 = hx*hx

ny = 70 #number_intervals for y
ay = 0
by = 1
hy = (by-ay)/ny
hy2 = hy*hy

'''approximated coefficient 
[[(c_i-1,j-1),(c_i-1,j),(c_i-1,j+1)]
 [ (c_i,j-1) , (c_i,j) , (c_i,j+1) ]
 [(c_i+1,j-1),(c_i+1,j),(c_i+1,j+1)]]
'''
C = [[   0   , -1/hx2     ,   0   ],
     [ -1/hy2, 2/hx2+2/hy2, -1/hy2],
     [   0   , -1/hx2     ,   0   ]]

A = numpy.zeros(((nx-1)*(ny-1),(nx-1)*(ny-1)))
B = numpy.zeros((nx-1)*(ny-1))
for i in range(nx-1):
    for j in range(ny-1):    
        T = numpy.zeros((nx+1,ny+1))
        T[i:i+3,j:j+3] = C
        Aij = T[1:-1,1:-1]
        A[i*(ny-1)+j] = Aij.reshape(Aij.size)
        s = 0
        if i*j==0 or i==nx-2 or j==ny-2:
            BC = numpy.array([[bf(ax+(i+0)*hx,ay+(j+0)*hy),bf(ax+(i+0)*hx,ay+(j+1)*hy),bf(ax+(i+0)*hx,ay+(j+2)*hy)],
                              [bf(ax+(i+1)*hx,ay+(j+0)*hy),            0              ,bf(ax+(i+1)*hx,ay+(j+2)*hy)],
                              [bf(ax+(i+2)*hx,ay+(j+0)*hy),bf(ax+(i+2)*hx,ay+(j+1)*hy),bf(ax+(i+2)*hx,ay+(j+2)*hy)]])
            T[1:-1,1:-1] = numpy.zeros_like(T[1:-1,1:-1])
            s = numpy.sum(BC*T[i:i+3,j:j+3])
        
        B[i*(ny-1)+j] = f(ax+(i+1)*hx,ay+(j+1)*hy) - s

# Compare solution
##### default variable

Imax = 10000

x0 = zero vector

eps = 1e-3

weight = 1

## 1. Exact solution

In [3]:
B
x1 = numpy.zeros_like(B)
x1.shape

(4761,)

In [4]:
e = math.e
def ef(x,y):
    return math.sin(2*math.pi*x)*y*y*(1-y)*(1-y)
x1 = numpy.zeros_like(B)
for i in range(nx-1):
    for j in range(ny-1):
        x1[i*(ny-1)+j] = ef(ax+hx*(i+1),ay+hy*(j+1))

## 2. Solution from x = inv(A)*b

In [5]:
t = time.time()
x2 = (numpy.linalg.inv(A)).dot(B)
print(time.time()-t)

1.5829095840454102


## 3.Solution by weighted Jacobi iteration

In [6]:
t = time.time()
x3 = LinSys.wJacobi(A,B)
print(time.time()-t)

max time ~ 112.22446894645691
solution dosen't change
7.254476308822632


## 4.Solution by SOR

In [7]:
t = time.time()
x4 = LinSys.SOR(A,B,weight=1)
print(time.time()-t)

max time ~ 162.79338026046753
solution dosen't change
9.441317081451416


## 5.Solution by gradiant method

In [8]:
t = time.time()
x5 = LinSys.GM(A,B)
print(time.time()-t)

max time ~ 109.35931539535522
solution dosen't change
33.071080923080444


## 6.Solution by conjugate gradiant method

In [9]:
t = time.time()
x6 = LinSys.CG(A,B,eps=1e-10)
print(time.time()-t)

solution converges
0.4432554244995117


# Compare solution with exact solution

In [10]:
print('Inverse \t\t',numpy.linalg.norm(x1-x2))
print('weighted Jacobi \t',numpy.linalg.norm(x1-x3))
print('SOR \t\t\t',numpy.linalg.norm(x1-x4))
print('gradiant method \t',numpy.linalg.norm(x1-x5))
print('conjugate gradiant \t',numpy.linalg.norm(x1-x6))

Inverse 		 0.0014161180076455668
weighted Jacobi 	 0.39448233699992513
SOR 			 0.19704366970791637
gradiant method 	 0.7886798296989036
conjugate gradiant 	 0.0014161180076523682
